In [2]:
import pandas as pd
import csv
import yfinance as yf

#Load Data

#Insider Data Amazon
insider_df = pd.read_csv('C:/Users/trist/code/Data/Finviz_insider/Data/AMZN.csv')

# Stock Data Amazon
ticker_symbol = 'AMZN'
start_date = '2023-01-01'
end_date = '2023-11-30'
amazon_df = yf.download(ticker_symbol, start=start_date, end=end_date)

# insider_df: name='Date', dtype= 'object'
# amazon_df: name='Date', dtype='datetime64[ns]' 


[*********************100%%**********************]  1 of 1 completed


## Todo

-> ich muss das SEC Date als bezugsdate verwenden, damit wurde Transaktion veröffentlicht

-> Uhrzeit muss ich berücksichtigen

In [3]:
#Enrich df with weekdays and ticker

insider_df['Date'] = pd.to_datetime(insider_df['Date'])
insider_df['Weekday_insider'] = insider_df['Date'].dt.day_name()

amazon_df['Weekday_stocks'] = amazon_df.index.day_name()
amazon_df['Ticker'] = ticker_symbol


In [4]:
insider_df.head()

,insider,title,transaction_date,transaction_type,cost,num_shares,value_shares,total_num_shares_insider_owns,SEC_date,Date,Weekday_insider
0,Zapolsky David,Senior Vice President,Nov 22,Sale,145.00,"18,440","2,673,800","53,220",Nov 24 04:36 PM,2023-11-22,Wednesday
1,Selipsky Adam,CEO Amazon Web Services,Nov 22,Sale,144.68,500,"72,340","121,060",Nov 24 04:23 PM,2023-11-22,Wednesday
2,Reynolds Shelley,Vice President,Nov 21,Option Exercise,0.00,"3,120",0,"122,900",Nov 24 04:16 PM,2023-11-21,Tuesday
3,Selipsky Adam,CEO Amazon Web Services,Nov 21,Option Exercise,0.00,"7,760",0,"125,164",Nov 24 04:23 PM,2023-11-21,Tuesday
4,Zapolsky David,Senior Vice President,Nov 21,Option Exercise,0.00,"16,800",0,"78,380",Nov 24 04:36 PM,2023-11-21,Tuesday


In [4]:
#Merge
insider_df = insider_df.sort_values('Date')
amazon_df = amazon_df.sort_values('Date')

merged_df = pd.merge_asof(insider_df, amazon_df, on='Date', direction='forward')

In [10]:
#Creating KPIs
small_df = merged_df[['Date', 'insider', 'Open', 'Close']]
stock_df_copy = small_df.copy()

# Calculate the daily percentage change using .loc on the copied DataFrame
stock_df_copy.loc[:, 'DailyChange'] = (stock_df_copy['Close'] - stock_df_copy['Open']) / stock_df_copy['Open'] * 100

# Calculate a 3, 7, 30 day percentage change 
stock_df_copy['3DayChange'] = 0.0
stock_df_copy['WeekAfterChange'] = 0.0
stock_df_copy['MonthAfterChange'] = 0.0

counter = 0  # Initialize the counter

from pandas.tseries.offsets import BDay
for index, row in stock_df_copy.iterrows():
    insider_date = row['Date']

    #Update 3DayChange Col
    threedays_after_date = insider_date + BDay(3)  # Use business day offset to add 3 weekdays
    threedays_after_date = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= threedays_after_date)]

    'Motiz: Feiertage werden nicht berücksichtigt in threedays_after_date -> daher falsche Listenlänge, Problem vielleicht zu lösen mit Wochenentagen und oder  forschleife?'

    
    if len(threedays_after_date) < 3: 
        stock_df_copy.at[index, '3DayChange'] = None
    else:
        percentage_change = (threedays_after_date['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
        stock_df_copy.at[index, '3DayChange'] = percentage_change


    # #Update WeekAfterChange Col
    # week_after_date = insider_date + pd.Timedelta(days=7)
    # week_after_data = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= week_after_date)]
    # if len(week_after_data) <= 7:
    #     stock_df_copy.at[index, 'WeekAfterChange'] = None
    # else:
    #     percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
    #     stock_df_copy.at[index, 'WeekAfterChange'] = percentage_change

    # #Update MonthAfterChange Col
    # Month_After_Change = insider_date + pd.Timedelta(days=30)
    # Month_After_Change = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= Month_After_Change)]
    # if len(Month_After_Change) <= 30:
    #     stock_df_copy.at[index, 'MonthAfterChange'] = None
    # else:
    #     percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
    #     stock_df_copy.at[index, 'MonthAfterChange'] = percentage_change

stock_df_copy.head()

4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
4
4
4
4
4
4
4
3
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3


,Date,insider,Open,Close,DailyChange,3DayChange,WeekAfterChange,MonthAfterChange
0,2023-03-21,Selipsky Adam,98.139999,100.610001,2.516814,-2.464967,0.0,0.0
1,2023-04-03,Herrington Douglas J,102.300003,102.410004,0.107527,-0.341769,0.0,0.0
2,2023-04-04,Selipsky Adam,102.750000,103.949997,1.167880,-1.818181,0.0,0.0
3,2023-04-05,Selipsky Adam,103.910004,101.099998,-2.704268,1.058358,0.0,0.0
4,2023-04-19,Selipsky Adam,101.580002,104.300003,2.677694,1.831252,0.0,0.0


In [20]:
# Specify the date range
start_date = '2023-04-04'
end_date = '2023-04-10'

#amazon_df['Date'] = pd.to_datetime(amazon_df['Date'])

filtered_df = amazon_df.loc[(amazon_df.index >= start_date) & (amazon_df.index <= end_date)]

print(filtered_df)



                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-04-04  102.750000  104.199997  102.110001  103.949997  103.949997   
2023-04-05  103.910004  103.910004  100.750000  101.099998  101.099998   
2023-04-06  100.750000  102.379997   99.800003  102.059998  102.059998   
2023-04-10  100.959999  102.199997   99.570000  102.169998  102.169998   

              Volume Weekday_stocks Ticker  
Date                                        
2023-04-04  48662500        Tuesday   AMZN  
2023-04-05  45175400      Wednesday   AMZN  
2023-04-06  43808000       Thursday   AMZN  
2023-04-10  37261200         Monday   AMZN  


In [84]:
from datetime import datetime
insider_date_str  = '2023-04-04'
insider_date = datetime.strptime(insider_date_str, '%Y-%m-%d')
print(insider_date)
threedays_after_date = insider_date + pd.Timedelta(days=3)
print(threedays_after_date)

threedays_after_date = amazon_df[(amazon_df.index >= insider_date) & (amazon_df.index <= threedays_after_date)]

if len(threedays_after_date) != 3:
    print("nicht genug Daten")
else:
    print(threedays_after_date)
# percentage_change = (threedays_after_date['Close'].iloc[-1] - row['Close']) / row['Close'] * 100
# print(percentage_change)





2023-04-04 00:00:00
2023-04-07 00:00:00
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-04-04  102.750000  104.199997  102.110001  103.949997  103.949997   
2023-04-05  103.910004  103.910004  100.750000  101.099998  101.099998   
2023-04-06  100.750000  102.379997   99.800003  102.059998  102.059998   

              Volume Weekday_stocks Ticker  
Date                                        
2023-04-04  48662500        Tuesday   AMZN  
2023-04-05  45175400      Wednesday   AMZN  
2023-04-06  43808000       Thursday   AMZN  


In [38]:
import pandas as pd

# Your existing DataFrame
data = {'Date': ['2023-03-21', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-19'],
        'insider': ['Selipsky Adam', 'Herrington Douglas J', 'Selipsky Adam', 'Selipsky Adam', 'Selipsky Adam'],
        'Open': [98.139999, 102.300003, 102.750000, 103.910004, 101.580002],
        'Close': [100.610001, 102.410004, 103.949997, 101.099998, 104.300003],
        'DailyChange': [2.516814, 0.107527, 1.167880, -2.704268, 2.677694]}

df = pd.DataFrame(data)

df.head()

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Create a new column for the week after insider trade
df['WeekAfterChange'] = 0.0  # Initialize the column with zeros

# Loop through each insider trade and calculate the percentage change for the week after
for index, row in df.iterrows():
    insider_date = row['Date']
    week_after_date = insider_date + pd.Timedelta(days=7)

    # Find the stock data for the week after
    week_after_data = df[(df['Date'] >= insider_date) & (df['Date'] <= week_after_date)]

    # Calculate the percentage change for the week after
    percentage_change = (week_after_data['Close'].iloc[-1] - row['Close']) / row['Close'] * 100

    # Update the 'WeekAfterChange' column
    df.at[index, 'WeekAfterChange'] = percentage_change

# # Display the updated DataFrame
# print(df)
